<a href="https://colab.research.google.com/github/andygma567/LLM-experiments/blob/main/OpenAlpaca_3BT_test_with_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This works kind of. It needs tuning or better prompting.

High RAM:
CPU 25 GB, GPU 16 GB, 5.45 credits/hour

Standard RAM:
CPU 12 GB, GPU 16 GB, 5.36 credits/hour



In [ ]:
!pip install -q transformers accelerate
# datasets
!pip install -q gradio
!pip install -q SentencePiece # required by Llama
!pip install -q xformers # required to use pipelines with OpenAlpaca

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.4 MB/s eta 0:00:

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import LlamaConfig, LlamaForCausalLM, LlamaTokenizer, LlamaModel
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import torch

# I want to time how long it takes to load everything
from timeit import default_timer as timer

# Check the device map

In [ ]:
# # the previewed version of OpenAlpaca
drive_checkpoint = r'/content/drive/MyDrive/openalpaca_7b_preview_3bt'
HF_checkpoint = "openllmplayground/openalpaca_7b_preview_3bt"

# config = AutoConfig.from_pretrained(drive_checkpoint)
# # config = LlamaConfig()

# with init_empty_weights():
#     model = AutoModelForCausalLM.from_config(config)

# # I think this is the method called by device_map='auto'
# my_drive_device_map = infer_auto_device_map(model)

In [ ]:
# config = AutoConfig.from_pretrained(HF_checkpoint)

# with init_empty_weights():
#     model = AutoModelForCausalLM.from_config(config)

# hf_hub_device_map = infer_auto_device_map(model)

In [ ]:
# print(my_drive_device_map == hf_hub_device_map)

# An attempt at loading a model

## Load the tokenizer

In [ ]:
# Takes 5.21 minutes to load from drive
# Loading from online takes 3.66 minutes
start = timer()

tokenizer = AutoTokenizer.from_pretrained(
    drive_checkpoint,
    use_fast=True,
    )
# I tried testing this but it didn't work for me
# tokenizer.bos_token_id, tokenizer.eos_token_id = 1,2 # see https://github.com/openlm-research/open_llama#preview-weights-release-and-usage

end = timer()
minutes = (end - start)/60
print(f'Elapsed minutes: {minutes:.2f}')

Elapsed minutes: 5.00


## Load the model

In [ ]:
# Takes 6 min to load the model from my drive
start = timer()

model = AutoModelForCausalLM.from_pretrained(
    drive_checkpoint,
    device_map="auto",
    offload_folder="offload",
    offload_state_dict = True,
    torch_dtype=torch.float16, # I don't think every Llama model supports half precision
    )

end = timer()
minutes = (end - start)/60
print(f'Elapsed minutes: {minutes:.2f}')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Elapsed minutes: 10.19


In [ ]:
instruction = r'What is an alpaca? How is it different from a llama?'
prompt_no_input = f'### Instruction:\n{input}\n\n### Response:'
tokens = tokenizer(prompt_no_input)

# Do I need to specify the return_tensors as pt?
# I think so
# The model can only accept tensors and if not specified then a dictionary (of numpy arrays?) is returned
# I just need to pass the input_ids as tensors to the model
# I think when there's only one sentence (no batching) then there's no need
# for padding and therefore no need for passing in attention masks too

In [ ]:
# print(output.keys())
# I think we only decode the logits
# output[0] == output['logits']

In [ ]:
# Maybe the max length is too small...
# Maybe I need to send the tokens to cuda? - this helped but it didn't fix things

# The issue was that it really needed the bos and eos tokens included in the input...
# Strangely the other 7BT OpenAlpaca doens't need this token stuff

# Everything works fine when it's not interpretting the input as being part-tensor
# I think this has something to do with how colab is caching variables...

In [ ]:
# setting the variable to result to the same variable is somehow different
input_ids = tokens["input_ids"] + [2] + [1]

In [ ]:
# LongTensor means it's a long data type so the ints are correct...
model_input = torch.IntTensor(input_ids).unsqueeze(0).to('cuda')

In [ ]:
# Check the input
# If there is a tensor appearing in here then colab is caching variables that it shouldn't be
print(tokenizer.decode(model_input.squeeze()))

<s> ### Instruction:
What is an alpaca? How is it different from a llama?

### Response:</s><s>


In [ ]:
gen_config = model.generation_config
print(gen_config)

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.29.2"
}



In [ ]:
with torch.no_grad():
  output = model.generate(
      model_input,
      max_length=512,
      # top_p=0.9,
      do_sample=True,
      )

In [ ]:
print(tokenizer.decode(output.squeeze()))

<s> ### Instruction:
What is an alpaca? How is it different from a llama?

### Response:</s><s> An Alpaca is a type of camelid native to the Andes mountains in South America. Alpacas have long legs, a long, curly mane, and a soft belly. They can weigh up to 130 pounds. Alpacas are sometimes referred to as “little lambs” because they have a wool that is very similar to sheep wool. There are many types of alpacas, including the suri breed which has unique longer legs and a single horn on their forehead.</s>


In [ ]:
def use_OpenAlpaca(input: str) -> str:
  # I think this prompt format may be important.
  # For example I get non-sense answers when I don't use this format
  # This is the prompt for the 3B token model
  prompt_no_input = f'### Instruction:\n{input}\n\n### Response:'
  tokens = tokenizer(prompt_no_input)

  # I tried to set the bos, eos tokenizer values but it didn't work for me
  bos_token_id, eos_token_id = 1, 2
  tokens = tokens["input_ids"] + [eos_token_id] + [bos_token_id]

  # They use torch.unsqueeze along the 0 dim because they didn't form their
  # tensor into batches for inputting to the model
  tokens = torch.LongTensor(tokens[-1024:]).unsqueeze(0).cuda()

  # They should have really just modified the generation_config
  instance = {'input_ids': tokens,
              'top_k': 50, # This is the default value
              'top_p': 0.9,
              'generate_len': 128}

  length = len(tokens[0])
  # This set up is for multinomial sampling according to the HF GenerationConfig docs
  # Here they are overriding the GenerationConfig of the model by passing in kwargs
  with torch.no_grad():
      rest = model.generate(
              input_ids=tokens,
              max_length=length+instance['generate_len'],
              use_cache=True,
              do_sample=True,
              top_p=instance['top_p'],
              # top_k=instance['top_k']
          )

  output = rest[0][length:]
  string = tokenizer.decode(output, skip_special_tokens=False)
  string = string.replace('<s>', '').replace('</s>', '').strip()
  return string

In [ ]:
# same prompt as provided in https://crfm.stanford.edu/2023/03/13/alpaca.html
instruction = r'What is an alpaca? How is it different from a llama?'
'''
instruction = r'Write an e-mail to congratulate new Standford admits and mention that you are excited about meeting all of them in person.'
instruction = r'What is the capital of Tanzania?'
instruction = r'Write a well-thought out abstract for a machine learning paper that proves that 42 is the optimal seed for training neural networks.'
'''

output = use_OpenAlpaca(input=instruction)
print(f'[!] Generation results: {output}')

[!] Generation results: An alpaca is a small, warm blooded, herbivorous mammal closely related to the camel. Alpacas are native to the Andes Mountains in South America. Alpacas are smaller than llamas and are often called lanky. Their coats come in many colors including white, black, tan, silver and brown. They have large, distinctive eyes that give them a solemn, gentle expression. Alpacas are mostly quiet and are considered quite smart. Alpacas are usually quiet and often observed relaxing in groups. Alpacas are mostly quiet and often observed relaxing


# An attempt at using a pipeline

For some reason, the pipelines aren't passing the input to the tokenizer and the model

In [ ]:
# # Set up an inference pipeline...
# from transformers import pipeline

# # In a lower RAM situation I can try to use accelerate with the auto device mapping,
# # offloading, and off load state dictionary
# gen = pipeline(
#     task="text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     # use_fast=True,
#     device='cuda:0', # alternatively cuda:0
#     )

In [ ]:
# Does the textgeneration pipeline append bos and eos tokens? Idk

# instruction=r'What is an alpaca?'

# prompt_no_input = f'### Instruction:\n{instruction}\n\n### Response:'

# generated_text=gen(prompt_no_input)
# print(generated_text)

# Set up a gradio demo

In [ ]:
import gradio as gr

# We instantiate the Textbox class
input_textbox = gr.Textbox(placeholder="Please enter an instruction", lines=2)

demo = gr.Interface(
    title=drive_checkpoint, # the path where the model is
    fn=use_OpenAlpaca,
    inputs=input_textbox,
    outputs="text",
    examples=[["What is an alpaca? How is it different from a llama?"],["How does a car work?"]],
    cache_examples=True,
    )

Caching examples at: '/content/gradio_cached_examples/15'


In [ ]:
demo.launch(share=True) #, debug=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a8529628f21162fda3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
